# Testing Notebook

I am going to run some functionality tests for componants of the final programme.

## Fetch Book Info. from ISBN

Creating a class that initializes with an ISBN and creates a Book object with details for the Library GUI.

LibraryThing API key ( I didn't end up using this):

Token Name: LibraryScanner

Token: 02b372f198a5ec0ee4e0dab3f2dc99f2

In [15]:
import requests

class Book:
    def __init__(self, isbn):
        self.isbn = isbn
        self.title = None
        self.author = None
        self.published_date = None
        self.cover_image_url = None

    def get_data(self):
        ''' Uses the 'isbn' class variable to query the Google Books catalogue'''
        api_url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{self.isbn}'

        try:
            response = requests.get(api_url)
            response.raise_for_status()
            data = response.json()

            # Extract relevant information from the API response
            if 'items' in data and data['items']:
                book_info = data['items'][0]['volumeInfo']
                self.title = book_info.get('title', 'N/A')
                self.author = ', '.join(book_info.get('authors', ['N/A']))
                self.published_date = book_info.get('publishedDate', 'N/A')

                # Choose the largest available cover image size
                # https://stackoverflow.com/questions/10721886/how-to-get-the-extra-large-cover-image-from-google-book-api
                image_links = book_info.get('imageLinks', {})
                sizes = ['extraLarge', 'large', 'medium', 'small', 'thumbnail', 'smallThumbnail']
                finding_biggest = True
                i = 0
                while finding_biggest:
                    size = sizes[i]
                    if size in image_links:
                        self.cover_image_url = image_links[size]
                        # The api will just give you the biggest image it has up to the number you set. 
                        # You can also set the height if you need to using e.g., '&fife=h900' 
                        # and both with e.g., '&fife=w800-h900'
                        self.cover_image_url = self.cover_image_url +'&fife=w600'
                        finding_biggest = False
                    i += 1
                    if i > len(sizes):  # cover image not found
                        self.cover_image_url = None
                        finding_biggest = False

        except requests.exceptions.RequestException as e:
            print(f"Error retrieving data from Google Books API: {e}")

    def display_info(self):
        print(f"Title: {self.title}")
        print(f"Author: {self.author}")
        print(f"Published Date: {self.published_date}")
        print(f"Cover Image URL: {self.cover_image_url}")

# Example usage:
isbn_to_query = '9780141030012'
book_instance = Book(isbn_to_query)
book_instance.get_data()
book_instance.display_info()



Title: Things I Want My Daughters to Know
Author: Elizabeth Noble
Published Date: 2008-09-04
Cover Image URL: http://books.google.com/books/content?id=e7iqjRF-dygC&printsec=frontcover&img=1&zoom=1&source=gbs_api&fife=w600


## Get ISBN from Barcode

## File Architecture

## GUI